In [1]:
import os
import pandas as pd
from pandas import Series, DataFrame
import country_converter as coco

Data sources:
- https://www.fao.org/faostat/en/#data/PP

Find file path

In [2]:
## Get current working directory
print("Current working directory:", os.getcwd())

## Find path to raw data (for the raw files)
raw_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'raw_data'))

## Find path to folder for saving cleaned csv
cleaned_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'cleaned_data'))


## Find path to folder for inflation
inflation_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'inflation'))

Current working directory: /Users/jennagreene/Documents/GitHub/HATCH_data/reading_files


In [3]:

target_inflation_file = "/A001RG3A086NBEA.xls"
target_inflation_filepath = inflation_data_path + target_inflation_file

target_file = "/cost data/FAOSTAT_data_en_8-14-2023.csv"
target_filepath = raw_data_path + target_file

In [4]:
## adjusting for inflation from 2015 USD to 2022
nipa = pd.read_excel(target_inflation_filepath, header=10)

year_list = []
for x in nipa['observation_date']:
    x = int(str(x)[:4])
    year_list.append(x)
nipa['Year'] = year_list
nipa.set_index('Year', drop=True, inplace=True)
nipa.drop(columns='observation_date', inplace=True)
nipa = nipa.transpose()

infl_2015_2022 = float(nipa[2022]/nipa[2015])
infl_2015_2022

/var/folders/cy/xd4mmw610b3cr6tjgv0h8hlr0000gn/T/ipykernel_29393/512181975.py:13: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  infl_2015_2022 = float(nipa[2022]/nipa[2015])


1.2154282765737874

In [5]:
faostat = pd.read_csv(target_filepath, usecols=[3,7,9,13], dtype={'Year': str})
# adjust for inflation
faostat['Value'] = faostat['Value'] * infl_2015_2022
faostat

,Area,Item,Year,Value
0,Albania,Maize (corn),1993,295.592157
1,Albania,Maize (corn),1994,254.389138
2,Albania,Maize (corn),1995,222.909546
3,Albania,Maize (corn),1996,302.398555
4,Albania,Maize (corn),1997,285.625645
...,...,...,...,...
5532,North Macedonia,Soya beans,2007,522.269530
5533,North Macedonia,Soya beans,2008,691.457147
5534,North Macedonia,Soya beans,2010,631.414990
5535,North Macedonia,Soya beans,2012,989.844788


In [6]:
def read_fao(crop, tech_name): 
    df = faostat[faostat['Item']==crop]
    df = df.pivot(index='Area', columns='Year', values='Value')
    df['Unit'] = '2022 USD/tonne'
    df['Metric'] = 'Price'
    df['Spatial Scale'] = 'National'
    df['Technology Name'] = tech_name
    df['Data Source'] = 'FAOSTAT'
    # drop Belgium-Luxembourg (we're not interested in subnational and this is like a province in Luxembourg)
    if 'Belgium-Luxembourg' in df.index:
        df.drop(['Belgium-Luxembourg'], inplace=True)
    # drop People's Democratic Republic of Ethiopia (only 2 years of data)
    if 'Ethiopia PDR' in df.index:
        df.drop(['Ethiopia PDR'], inplace=True)
    df['Country Name'] = df.index
    iso = []
    for country in df.index:
        iso2 = coco.convert(names=country, to='iso2')
        iso.append(iso2)
    df['Country Code'] = iso
    df['ID'] = df['Technology Name'] + '_' + df['Metric'] + '_' + df['Country Code']
    df.set_index('ID', drop=True, inplace=True)

    # Define the path and file name for saving the processed DataFrame
    output_file = 'fao_' + tech_name +'.csv'
    output_file_path = os.path.join(cleaned_data_path, output_file)
    
    # Save the processed DataFrame to a CSV file
    df.to_csv(output_file_path)
    
    return df

In [7]:
corn = read_fao('Maize (corn)', 'Corn Production')
corn

Year,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2019,2020,2021,Unit,Metric,Spatial Scale,Technology Name,Data Source,Country Name,Country Code
ID,,,,,,,,,,,,,,,,,,,,,
Corn Production_Price_AL,NaN,NaN,295.592157,254.389138,222.909546,302.398555,285.625645,298.509185,282.465531,245.273426,...,336.673633,344.209288,433.786352,2022 USD/tonne,Price,National,Corn Production,FAOSTAT,Albania,AL
Corn Production_Price_DZ,210.512178,634.575103,593.493627,395.257276,408.019272,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2022 USD/tonne,Price,National,Corn Production,FAOSTAT,Algeria,DZ
Corn Production_Price_AO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2022 USD/tonne,Price,National,Corn Production,FAOSTAT,Angola,AO
Corn Production_Price_AG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2113.508230,2113.508230,3099.828277,2022 USD/tonne,Price,National,Corn Production,FAOSTAT,Antigua and Barbuda,AG
Corn Production_Price_AR,114.614886,74.870382,131.387797,130.172368,139.774252,184.866641,132.603225,117.896543,108.173117,99.665119,...,NaN,NaN,NaN,2022 USD/tonne,Price,National,Corn Production,FAOSTAT,Argentina,AR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Corn Production_Price_NZ,175.264757,163.232018,174.049329,198.236352,219.384804,232.268344,200.910294,159.342647,157.641047,133.575568,...,343.480031,334.242776,373.987281,2022 USD/tonne,Price,National,Corn Production,FAOSTAT,New Zealand,NZ
Corn Production_Price_NI,126.890712,152.900877,187.297497,261.924794,324.276264,311.028096,263.383308,229.837487,193.131553,255.361481,...,384.075335,368.517853,402.428302,2022 USD/tonne,Price,National,Corn Production,FAOSTAT,Nicaragua,NI
Corn Production_Price_NE,NaN,NaN,171.740015,140.138880,167.972188,168.701445,170.767673,171.010759,157.884133,143.420537,...,NaN,NaN,NaN,2022 USD/tonne,Price,National,Corn Production,FAOSTAT,Niger,NE


In [8]:
soy = read_fao('Soya beans', 'Soybean Production')
soy

Year,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2019,2020,2021,Unit,Metric,Spatial Scale,Technology Name,Data Source,Country Name,Country Code
ID,,,,,,,,,,,,,,,,,,,,,
Soybean Production_Price_AL,NaN,NaN,NaN,NaN,NaN,581.582430,408.019272,403.400645,529.683643,592.035114,...,720.384340,676.385836,806.436662,2022 USD/tonne,Price,National,Soybean Production,FAOSTAT,Albania,AL
Soybean Production_Price_AO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2022 USD/tonne,Price,National,Soybean Production,FAOSTAT,Angola,AO
Soybean Production_Price_AR,210.998349,203.705779,266.421878,274.929876,263.869479,330.718034,353.811171,257.792337,203.098065,218.898633,...,NaN,NaN,NaN,2022 USD/tonne,Price,National,Soybean Production,FAOSTAT,Argentina,AR
Soybean Production_Price_AU,335.093576,314.188209,306.652554,391.854076,358.551342,380.429051,379.821336,316.862152,319.900722,242.356398,...,NaN,NaN,NaN,2022 USD/tonne,Price,National,Soybean Production,FAOSTAT,Australia,AU
Soybean Production_Price_AT,NaN,586.201058,553.870666,542.688725,269.217363,294.619814,283.437874,243.936455,231.052915,202.854979,...,427.223039,487.265196,710.174742,2022 USD/tonne,Price,National,Soybean Production,FAOSTAT,Austria,AT
Soybean Production_Price_AZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2022 USD/tonne,Price,National,Soybean Production,FAOSTAT,Azerbaijan,AZ
Soybean Production_Price_BD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,441.443550,1181.031656,NaN,2022 USD/tonne,Price,National,Soybean Production,FAOSTAT,Bangladesh,BD
Soybean Production_Price_BZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2022 USD/tonne,Price,National,Soybean Production,FAOSTAT,Belize,BZ
Soybean Production_Price_BT,NaN,NaN,NaN,NaN,NaN,267.515764,331.325748,303.370898,338.739861,378.605908,...,NaN,NaN,NaN,2022 USD/tonne,Price,National,Soybean Production,FAOSTAT,Bhutan,BT


In [9]:
cotton = read_fao('Cotton seed', 'Cotton Seed Production')
cotton

Year,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2019,2020,2021,Unit,Metric,Spatial Scale,Technology Name,Data Source,Country Name,Country Code
ID,,,,,,,,,,,,,,,,,,,,,
Cotton Seed Production_Price_AZ,NaN,NaN,NaN,138.315738,109.023916,67.577812,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2022 USD/tonne,Price,National,Cotton Seed Production,FAOSTAT,Azerbaijan,AZ
Cotton Seed Production_Price_BO,54.694272,42.418447,61.014499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2022 USD/tonne,Price,National,Cotton Seed Production,FAOSTAT,Bolivia (Plurinational State of),BO
Cotton Seed Production_Price_BI,NaN,NaN,NaN,NaN,NaN,NaN,53.114216,68.914783,73.290325,70.859469,...,NaN,NaN,NaN,2022 USD/tonne,Price,National,Cotton Seed Production,FAOSTAT,Burundi,BI
Cotton Seed Production_Price_CN,101.366718,115.465686,127.012255,142.812822,402.428302,520.689474,355.148142,355.391228,359.280599,441.686636,...,434.150980,487.872910,NaN,2022 USD/tonne,Price,National,Cotton Seed Production,FAOSTAT,"China, mainland",CN
Cotton Seed Production_Price_CI,85.323065,101.002090,98.692776,78.759752,102.217518,71.224097,49.954102,49.467931,45.457018,51.169530,...,NaN,NaN,NaN,2022 USD/tonne,Price,National,Cotton Seed Production,FAOSTAT,Côte d'Ivoire,CI
Cotton Seed Production_Price_DO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,30.385707,30.021078,2022 USD/tonne,Price,National,Cotton Seed Production,FAOSTAT,Dominican Republic,DO
Cotton Seed Production_Price_EG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.783824,15.679025,13.977425,...,NaN,NaN,NaN,2022 USD/tonne,Price,National,Cotton Seed Production,FAOSTAT,Egypt,EG
Cotton Seed Production_Price_SV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2022 USD/tonne,Price,National,Cotton Seed Production,FAOSTAT,El Salvador,SV
Cotton Seed Production_Price_GM,403.643731,396.229618,386.749278,367.910139,432.935552,NaN,NaN,NaN,373.258024,427.709211,...,NaN,NaN,NaN,2022 USD/tonne,Price,National,Cotton Seed Production,FAOSTAT,Gambia,GM


In [10]:
milk = read_fao('Raw milk of cattle', 'Cow\'s Milk Production')
milk

Year,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2019,2020,2021,Unit,Metric,Spatial Scale,Technology Name,Data Source,Country Name,Country Code
ID,,,,,,,,,,,,,,,,,,,,,
Cow's Milk Production_Price_AL,NaN,NaN,418.593498,NaN,262.289422,232.632972,285.625645,282.465531,317.834494,346.761687,...,454.448633,493.220795,538.313184,2022 USD/tonne,Price,National,Cow's Milk Production,FAOSTAT,Albania,AL
Cow's Milk Production_Price_DZ,460.525774,556.666151,624.730134,624.000877,561.041692,488.359082,505.496620,537.948555,474.624742,NaN,...,488.845253,472.072343,463.564345,2022 USD/tonne,Price,National,Cow's Milk Production,FAOSTAT,Algeria,DZ
Cow's Milk Production_Price_AO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2022 USD/tonne,Price,National,Cow's Milk Production,FAOSTAT,Angola,AO
Cow's Milk Production_Price_AG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2378.593137,2378.593137,2378.593137,...,NaN,NaN,NaN,2022 USD/tonne,Price,National,Cow's Milk Production,FAOSTAT,Antigua and Barbuda,AG
Cow's Milk Production_Price_AR,NaN,184.015841,200.788751,208.081321,221.207946,NaN,224.975774,211.606063,171.496930,188.512926,...,358.915970,NaN,NaN,2022 USD/tonne,Price,National,Cow's Milk Production,FAOSTAT,Argentina,AR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cow's Milk Production_Price_NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1366.992183,1392.759262,1565.228535,2022 USD/tonne,Price,National,Cow's Milk Production,FAOSTAT,New Caledonia,NC
Cow's Milk Production_Price_NZ,137.464938,176.237100,194.468524,193.860810,224.975774,270.675877,237.859314,180.248013,186.568240,170.646130,...,436.460294,463.442802,492.613080,2022 USD/tonne,Price,National,Cow's Milk Production,FAOSTAT,New Zealand,NZ
Cow's Milk Production_Price_NI,NaN,NaN,302.520098,297.536842,372.893395,373.014938,375.081166,367.302425,342.264603,334.972033,...,417.013442,372.650310,NaN,2022 USD/tonne,Price,National,Cow's Milk Production,FAOSTAT,Nicaragua,NI


In [11]:
sugar = read_fao('Sugar cane', 'Sugar Cane Production')
sugar[sugar.columns[20:30]]

Year,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
ID,,,,,,,,,,
Sugar Cane Production_Price_AR,44.363132,32.451935,30.993421,31.236507,NaN,NaN,NaN,NaN,NaN,NaN
Sugar Cane Production_Price_AU,47.644788,54.086558,48.130960,43.876961,36.584391,33.424278,40.959933,36.341305,32.938106,34.639706
Sugar Cane Production_Price_BD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sugar Cane Production_Price_BB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sugar Cane Production_Price_BZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sugar Cane Production_Price_BT,NaN,425.278354,728.041538,448.128406,416.770356,361.711455,839.860939,NaN,NaN,NaN
Sugar Cane Production_Price_BO,28.684107,29.291821,30.021078,360.374484,317.469866,118.018086,63.080728,67.820898,NaN,NaN
Sugar Cane Production_Price_BR,37.678277,38.650619,36.341305,32.816563,23.093137,24.916280,NaN,23.093137,22.363880,18.960681
Sugar Cane Production_Price_BI,NaN,NaN,54.694272,55.058901,NaN,NaN,49.224845,47.766331,46.064732,NaN
